In [1]:
import pandas as pd
import numpy as np
import glob
import ProgressBar as pb
dates = pd.read_pickle('./timecode.pcl')
wyears = np.load('./wyears.npy')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']
forcing_columns = ['P','Tmax','Tmin','W']

In [2]:
with np.load('./spl_er_L3_domain.npz') as data:
    lat = data['lat']
    lon = data['lon']
    indexer = data['indexer']
    regcode = data['rc']

In [3]:
print len(lat)
print len(lon)

9361
9361


In [4]:
i = 0
fl = '/Users/barnhatb/research/vic/fluxes/ascii/latitude.'+str(lat[i])+'/flux_snow_'+str(lat[i])+'_'+str(lon[i])
fr = '/Users/barnhatb/research/vic/forcing/ascii/latitude.'+str(lat[i])+'/Meteorology_Livneh_NAmerExt_15Oct2014_'+str(lat[i])+'_'+str(lon[i])

flux = pd.read_table(fl, sep='\t', names=fluxes_columns)
flux.index = pd.DatetimeIndex(dates)

flux = flux.loc['1988-01-01':'2013-12-31',:]

In [5]:
n = len(flux)
print n

m = len(lat)
print m

q = m*n
print q
q += 1000000


# preallocate
days = np.zeros([q]) # preallocate for days
days[:] = np.NaN # fill with NaNs
PET = days.copy() # preallocate for PET
P = days.copy() # preallocate for P
idx = days.copy() # preallocate for cell index
rc = days.copy() # preallocate for region code

strtct = 0 # initialize the indexer
ndct = strtct

p = pb.ProgressBar(q)

for i in xrange(m):

    # generate file names for Ullr
    fl = '/Users/barnhatb/research/vic/fluxes/ascii/latitude.'+str(lat[i])+'/flux_snow_'+str(lat[i])+'_'+str(lon[i])
    fr = '/Users/barnhatb/research/vic/forcing/ascii/latitude.'+str(lat[i])+'/Meteorology_Livneh_NAmerExt_15Oct2014_'+str(lat[i])+'_'+str(lon[i])
    
    # load the fluxes data
    flux = pd.read_table(fl, sep='\t', names=fluxes_columns)
    flux.index = pd.DatetimeIndex(dates)

    ## load the forcing data
    force = pd.read_table(fr, sep=' ', names=forcing_columns)
    force.index = pd.DatetimeIndex(dates)
    
    # grab only the days needed 
    flux = flux.loc['1988-01-01':'2013-12-31',:]
    force = force.loc['1988-01-01':'2013-12-31',:]
    
    ndct = strtct + len(flux) # compute the end indexer
    
    days[strtct:ndct] = flux.index.to_julian_date()
    PET[strtct:ndct] = flux.PEText.as_matrix() 
    P[strtct:ndct] = force.P.as_matrix()
    idx[strtct:ndct] = np.repeat(indexer[i],len(force.P))
    rc[strtct:ndct] = np.repeat(regcode[i],len(force.P))
    
    strtct = ndct+1 # compute the start indexer for the next iteration
    
    #for day in flux.index:
    #    days[ct] = day.to_julian_date()
    #    PET[ct] = flux.loc[day,'PEText']
    #    P[ct] = force.loc[day,'P']
    #    ct += 1
    p.animate(ndct)
    
    

9497
9361
88901417
[*****************99%******************]  88901280 of 89901417 complete


In [ ]:
len(flux.index.to_julian_date())

In [ ]:
len(days[strtct:ndct])

In [ ]:
ndct

In [ ]:
ndct-strtct

In [6]:
np.savez_compressed('./spl_L3_ecoregion_cals_2',days=days,PET=PET,P=P,idx=idx,rc=rc)

In [ ]:
!ls -lth